In [3]:
pip install PublicDataReader --upgrade

Note: you may need to restart the kernel to use updated packages.


In [4]:
import PublicDataReader as pdr
import pandas as pd

In [40]:
noin_data = pd.read_csv("../data/native_admdong_sum_2023061.csv")

In [5]:
busstop = pd.read_csv("../data/busstop.csv")
busstop["읍명동인구수"] = 0

### 주소를 법정동 코드 -> 행정동 코드로 변환 1번 방법



In [7]:
연결정보 = pdr.code_hdong_bdong()
연결정보 = 연결정보[연결정보["시도명"] == '경상남도']


for i in range(len(busstop["상세주소"])):
    for j in range(len(연결정보["동리명"])):
        if 연결정보["동리명"][j] in busstop["상세주소"][i]:
            for k in range(len(noin_data["admdong_cd"])):
                if noin_data["admdong_cd"][k] in 연결정보["행정동코드"][j]:
                    busstop["읍명동인구수"][i] = noin_data["0"]
                    break
            break



출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)


KeyError: '상세주소'

### 주소를 법정동 코드 -> 행정동 코드로 변환 2번 방법



In [ ]:
연결정보 = pdr.code_hdong_bdong()
연결정보 = 연결정보[연결정보["시도명"] == '경상남도']

# 연결정보 데이터에서 '법정동코드'와 '동리명'을 기준으로 사전 생성
동리명_법정동코드_매핑 = dict(zip(연결정보['법정동코드'], 연결정보['동리명']))

# noin_data 데이터에서 'admdong_cd'와 인구수를 기준으로 사전 생성
admdong_cd_인구수_매핑 = dict(zip(noin_data['admdong_cd'], noin_data['0']))

# busstop 데이터프레임에서 인구수를 업데이트
for i in range(len(busstop)):
    상세주소 = busstop.loc[i, '정류장명']  # 여기서는 '정류장명'이 상세주소로 사용됨
    for 법정동코드, 동리명 in 동리명_법정동코드_매핑.items():
        if 동리명 in 상세주소:
            if 법정동코드 in admdong_cd_인구수_매핑:
                busstop.loc[i, '읍명동인구수'] = admdong_cd_인구수_매핑[법정동코드]
                break

In [6]:
busstop

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,geometry,읍명동인구수
0,CWB379000566,구암고등학교,35.246296,128.593978,2023-10-16,420615.0,38010,경상남도 창원시,창원,POINT (1099529.3099473445 1695090.3215359906),0
1,CWB379000567,사격장입구,35.244923,128.683156,2023-10-16,107508.0,38010,경상남도 창원시,창원,POINT (1107644.8457543373 1695031.1020157929),0
2,CWB379000568,장애인복지회관입구,35.244960,128.673597,2023-10-16,119117.0,38010,경상남도 창원시,창원,POINT (1106775.0680812967 1695024.859684883),0
3,CWB379000570,장애인복지회관입구,35.244773,128.673243,2023-10-16,119118.0,38010,경상남도 창원시,창원,POINT (1106743.105090053 1695003.6802969584),0
4,CWB379000573,지귀상가,35.244778,128.659437,2023-10-16,119111.0,38010,경상남도 창원시,창원,POINT (1105486.9448440736 1694989.4555102403),0
...,...,...,...,...,...,...,...,...,...,...,...
18854,YSB5166,사과나무집,35.491970,128.989600,2023-10-16,5166.0,38100,경상남도 양산시,양산,POINT (1135114.8887175242 1722807.740568875),0
18855,YSB5167,이지더원3차,35.322630,129.027700,2023-10-16,5167.0,38100,경상남도 양산시,양산,POINT (1138861.2639738221 1704077.9474447218),0
18856,YSB5168,반도유보라6차,35.317890,129.027500,2023-10-16,5168.0,38100,경상남도 양산시,양산,POINT (1138851.1887177934 1703551.928831223),0
18857,YSB5169,양우내안애6차,35.318010,129.023400,2023-10-16,5169.0,38100,경상남도 양산시,양산,POINT (1138478.2608111338 1703559.5004212412),0
